In [14]:
from scipy.stats import norm
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
from datetime import date
def numOfDays(date1, date2):
    return (date2-date1).days

In [15]:
import ipywidgets as w
from ipycanvas import MultiCanvas
from ipywidgets import Image
from ipycanvas import hold_canvas
from IPython.display import display

In [16]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl

plt.style.use('dark_background')
plt.rcParams.update({'font.size': 16, "axes.facecolor":  "#2d2d2dff", 'legend.framealpha': 1.})


backend_ =  mpl.get_backend() 
mpl.use("Agg")

In [17]:
def f(parameterset):
    timeinit = time.time()
    frequency = parameterset["frequency"]
    if frequency == "Hourly":
        f = 1
    elif frequency == "Daily":
        f = 24
    elif frequency == "Weekly":
        f = 24*7
    else:
        raise ValueError("Unkown frequency")
        
    m = parameterset["m"]
    N = parameterset["N"]

    std_LT = parameterset["std_LT"]
    mean_demand = parameterset["mean_demand"] #Mean demand by time step
    std_demand = parameterset["std_demand"]#13 #Std of demand in one time step
    LT = parameterset["LT"] #Lead time (number of time steps)

    reorder = parameterset["reorderpoint"]
    reorder_quantity = parameterset["reorderquantity"]

    inventory = reorder_quantity #Initial inventory
    arrives = {}
    cycle = 0
    stockout = 0
    history2 = []
    orders = []
    totaldemands = []
    totalserveddemands = []
    wait = False #Waiting for a shipment
    total = 0
    satisfied = 0
    cycleservicelevel = []
    demands = []
    remainingtimetoarrival = 0
    timetoarrivals = []
    waits = []
    fillrate = []
    duration_transport = 1
    neg = 0
    neg2 = 0
    dailywaits = []
    dailyorders = []
    dailyhistory = []
    dailytimetoarrivals = []
    dailytotaldemands = []
    dailytotalserveddemands = []
    dailyfillrate = []
    dailycycleservicelevel = []
    
    for i in range(N):
        dailydemands = 0
        dailyorder = False
        for j in range(m):
            k = i*m+j
            
            order = False
            lastinventory = inventory
            demand = np.random.normal(mean_demand/m, std_demand/np.sqrt(m))
                
            dailydemands += demand
            total += demand

            if inventory < demand:
                if inventory > 0:
                    satisfied += inventory
                inventory = 0
            else:
                inventory -= demand
                satisfied += demand

            if wait == True:
                remainingtimetoarrival -= 1/duration_transport

            if (inventory <= reorder and (not wait) and (k%f ==0)):
                #Reorder, and wait for shipment
                duration_transport = int(m*np.random.normal(LT, std_LT))
                arrives[k+duration_transport] = reorder_quantity
                wait = True
                order = True
                dailyorder = True
                remainingtimetoarrival = 1
                
            if k in arrives:
                remainingtimetoarrival = 0
                wait = False #Stop waiting for a shipment
                cycle += 1
                if std_demand == 0 and std_LT == 0:
                    inventorycheck = lastinventory
                else:
                    inventorycheck = inventory

                if inventorycheck <= 0: #Check whether a stockout happened in this cycle
                    stockout += 1
                    inventory = arrives[k]
                else:
                    inventory += arrives[k]
            
            history2.append(inventory)
            orders.append(order)
            waits.append(wait)
            timetoarrivals.append(remainingtimetoarrival)
            totaldemands.append(total)
            totalserveddemands.append(satisfied)
            fill = satisfied/total if total > 0 else 1
            fillrate.append(fill)
            cyclesl = (cycle-stockout)/cycle if cycle > 0 else 1
            cycleservicelevel.append(cyclesl)
        demands.append(dailydemands)
        dailywaits.append(wait)
        dailyorders.append(dailyorder)
        dailyhistory.append(inventory)
        dailytimetoarrivals.append(remainingtimetoarrival)
        dailytotaldemands.append(total)
        dailytotalserveddemands.append(satisfied)
        fill = satisfied/total if total > 0 else 1
        dailyfillrate.append(fill)
        cyclesl = (cycle-stockout)/cycle if cycle > 0 else 1
        dailycycleservicelevel.append(cyclesl)
        
        
        
    results = {"totaldemands": totaldemands, "totalserveddemands": totalserveddemands, "cycleservicelevel": cycleservicelevel, "fillrate": fillrate, "orders": orders, "waits": waits, "timetoarrivals": timetoarrivals, "reorder": reorder, "stockout": stockout, "cycle": cycle, "satisfied": satisfied, "total": total, "cycle service level": (cycle-stockout)/cycle, "fill rate service level": satisfied/total}
    results["demands"] = demands
    results["history"] = history2
    results["dailyhistory"] = dailyhistory
    results["dailywaits"] = dailywaits
    results["dailyorders"] = dailyorders
    results["dailytimetoarrivals"] = dailytimetoarrivals
    results["dailytotaldemands"] = dailytotaldemands
    results["dailytotalserveddemands"] = dailytotalserveddemands
    results["dailyfillrate"] = dailyfillrate
    results["dailycycleservicelevel"] = dailycycleservicelevel
    results["meandemand"] = parameterset["mean_demand"]
    return results

In [18]:
timestepsperday = 24
simulatedyears = 10
simulateddays = numOfDays(date(2023, 1, 1), date(2023+10, 1, 1))
iday = pd.date_range(start=pd.Timestamp('2023-01-01 00:00:00'), periods = simulateddays*2, freq='1D')
i2day = pd.date_range(start=pd.Timestamp('2022-11-01 00:00:00'), periods = simulateddays*2, freq='1D')
jday = iday.strftime('%Y-%m-%d')
kday = i2day.strftime('%Y-%m-%d')

In [19]:
m = MultiCanvas(n_canvases=12, width=1711, height=1200)
def draw_square(canvas, x, y, width, color):
    canvas.fill_style = color
    canvas.fill_rect(x, y, width, width)
    
def draw_rectangle(canvas, x, y, width, height, color):
    canvas.fill_style = color
    canvas.fill_rect(x, y, width, height)
    
def draw_empty_square(canvas, x, y, width, color):
    canvas.stroke_style = color
    canvas.stroke_rect(x, y, width, width)
    
def draw_empty_rectangle(canvas, x, y, width, height, color):
    canvas.stroke_style = color
    canvas.stroke_rect(x, y, width, height)

def write_text(canvas, text, x, y):
    canvas.font = '10px sans-serif'
    canvas.fill_style = 'black'
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    canvas.fill_text(text, x, y)

In [20]:
# define a function which returns an image as numpy array from figure
def get_img_from_fig(fig, dpi=80):
    buf = io.BytesIO()
    # fig.tight_layout()
    fig.savefig(buf, facecolor=fig.get_facecolor(), format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mpl.pyplot.close()
    return img

In [21]:
res = f({"m": timestepsperday, "N": simulateddays,
         "mean_demand": 100, "std_demand": 10, "LT": 15, "std_LT": 0.5, "reorderpoint": 1553, "reorderquantity": 3000, "frequency": "Hourly"})

In [22]:
def draw_inventory(iteration):
    ref = iday[iteration - min(iteration, 60): iteration+1]
    y = res["dailyhistory"][iteration - min(iteration, 60): iteration+1]
    
    fig = plt.figure(facecolor="#2d2d2dff", figsize=(6,5))
    
    from mpl_toolkits.axes_grid1 import Divider, Size
    h = [Size.Fixed(1.7), Size.Fixed(2.5)]
    v = [Size.Fixed(1), Size.Fixed(3.5)]

    divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
    ax = fig.add_axes(divider.get_position(),
                      axes_locator=divider.new_locator(nx=1, ny=1))
    ax.set_xlim([pd.Timestamp(kday[iteration]), pd.Timestamp(jday[iteration + 30])])
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    fig.autofmt_xdate()
    ax.set_ylim(-100, np.max(res["dailyhistory"]))
    ax.set_xlabel("Date")
    ax.set_ylabel("Inventory level")
    ax.set_facecolor("#2d2d2dff")
    ax.axhline(0, linewidth = 3, color = 'tab:red')
    ax.plot(ref, y, linewidth = 3, color = 'tab:blue')
    
    return get_img_from_fig(fig)

In [23]:
def draw_demand(iteration):
    ref = iday[iteration - min(iteration, 60): iteration+1]    
    y = res["demands"][iteration - min(iteration, 60): iteration+1]
    fig = plt.figure(facecolor="#2d2d2dff", figsize=(6,5))
    
    from mpl_toolkits.axes_grid1 import Divider, Size
    h = [Size.Fixed(1.7), Size.Fixed(2.5)]
    v = [Size.Fixed(1), Size.Fixed(3.5)]

    divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
    ax = fig.add_axes(divider.get_position(),
                      axes_locator=divider.new_locator(nx=1, ny=1))
    ax.set_xlim([pd.Timestamp(kday[iteration]), pd.Timestamp(jday[iteration +30])])
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    fig.autofmt_xdate()
    ax.set_ylim(np.min(res["demands"])-5, np.max(res["demands"])+5)
    ax.set_xlabel("Date")
    ax.set_ylabel("Daily demand")
    ax.set_facecolor("#2d2d2dff")
    ax.plot(ref, y, linewidth = 3, color = 'tab:green')
    
    return get_img_from_fig(fig)

In [24]:
def display_function(iteration = 0):
    if iteration==0:
        m[1].draw_image(Image.from_file('background2.png'), 0, 0, 1711, 921)
        m[4].font = "32px sans-serif"
        m[4].fill_style = "white"
        m[5].font = "32px sans-serif"
        m[5].fill_style = "white"
        m[6].font = "26px sans-serif"
        m[6].fill_style = "white"
        m[7].font = "26px sans-serif"
        m[7].fill_style = "white"
    with hold_canvas():
        reorder = int(res['reorder'])
        res["dailytmporders"]=[]
        cutoff = 3
        tmp_i = cutoff
        for i in res["dailyorders"]:
            tmp_i+=1
            if tmp_i<cutoff or i:
                res["dailytmporders"].append(True)
            else:
                res["dailytmporders"].append(False)
            if i:
                tmp_i = 0
        m[3].clear()
        m[4].clear()
        m[6].clear()
        m[7].clear()
        m[5].clear()
        m[5].fill_text(str(jday[iteration]), 750, 50, max_width=200)
        if res["dailyhistory"][iteration] >= reorder:
            draw_rectangle(m[3], 773, 485, 100, 40, 'green')
        if res["dailyhistory"][iteration] < reorder and res["dailyhistory"][iteration]>0:
            draw_rectangle(m[3], 773, 485, 100, 40, 'orange')
        if res["dailyhistory"][iteration] ==0:
            draw_rectangle(m[3], 773, 485, 100, 40, 'red')
        m[4].fill_text(str(int(res["dailyhistory"][iteration])), 785, 515, max_width=100)
        m[2].put_image_data(draw_inventory(iteration), 590, 510)
        if res["dailytmporders"][iteration]:
            m[5].draw_image(Image.from_file("ordering.png"), 200, 130, width=600, height=150)
            m[5].fill_text("Order", 480, 260, max_width=100)
        if res["dailywaits"][iteration]:
            advance = res["dailytimetoarrivals"][iteration]
            m[6].draw_image(Image.from_file("delivery.png"), int(advance*250 + (1-advance)*680), 400, width=80, height=80)
        m[7].fill_text("Total demand: "+str(int(res["dailytotaldemands"][iteration])), 1250, 700, max_width=400)
        m[7].fill_text("Served demand: "+str(int(res["dailytotalserveddemands"][iteration])), 1250, 740, max_width=400)
        fillratesl = res["dailyfillrate"][iteration]*100
        m[7].fill_text("Fill rate service level: "+ f"{fillratesl:.1f}"+"%", 1250, 780, max_width=400)
        cyclesl = res["dailycycleservicelevel"][iteration]*100
        m[7].fill_text("Cycle service level: "+f"{cyclesl:.1f}"+"%", 1250, 820, max_width=400)
        m[8].put_image_data(draw_demand(iteration), 1145, 200)
        
m.clear()
m.on_client_ready(display_function)

In [25]:
maximum = simulateddays-1

meandemand = w.BoundedIntText(
    value=100,
    min=0,
    max=10000,
    step=1,
    description='Mean daily demand:',
    disabled=False,
    style={'description_width': 'initial'}
)
stddemand = w.BoundedFloatText(
    value=10,
    min=0,
    max=10000,
    step=0.01,
    description='Standard deviation of daily demand:',
    disabled=False,
    style={'description_width': 'initial'}
)
leadt = w.BoundedFloatText(
    value=15,
    min=0,
    max=1000,
    step=0.1,
    description='Mean lead time (days):',
    disabled=False,
    style={'description_width': 'initial'}
)
stdleadt = w.BoundedFloatText(
    value=0.5,
    min=0,
    max=1000,
    step=0.01,
    description='Standard deviation of lead time (days):',
    disabled=False,
    style={'description_width': 'initial'}
)
reorderstock = w.BoundedIntText(
    value=1553,
    min=0,
    max=10000,
    step=1,
    description='Reorder point:',
    disabled=False,
    style={'description_width': 'initial'}
)
reorderqty = w.BoundedIntText(
    value=3000,
    min=0,
    max=10000,
    step=1,
    description='Reorder quantity:',
    disabled=False,
    style={'description_width': 'initial'}
)
review = w.Dropdown(
    options=['Hourly', 'Daily', 'Weekly'],
    value='Hourly',
    description='Review period:',
    disabled=False,
    style={'description_width': 'initial'}
)

runsimulation = w.Button(
    description='Reload simulator with parameters above',
    disabled=False,
    button_style='info',#, 'info', 'warning', 'danger' or ''
    icon='refresh',
    layout=w.Layout(width='100%',
                              grid_area='middle',
                              padding="10px 200px 35px 200px")
)

step_picker = w.BoundedIntText(description="Day", min=0, max=maximum)
slider_picker = w.IntSlider(min=0, max=maximum)
play = w.Play(interval=150,
              value=0,
              min=0,
              max=maximum,
              step=1,
              description="Press play",
              disabled=False)
speed_picker = w.FloatSlider(description="Speed", min=0, max=1, value = 0.6)
out = w.interactive_output(display_function, {'iteration':slider_picker})

def on_button_clicked(runsimulation):
    results = f({"m": timestepsperday, "N": simulateddays, \
       "mean_demand": meandemand.value, "std_demand": stddemand.value, "LT": leadt.value, "std_LT": stdleadt.value, \
                 "reorderpoint": reorderstock.value, "reorderquantity": reorderqty.value, "frequency": review.value})
    res.clear()
    res.update(results)
    play.value = 0
    
runsimulation.on_click(on_button_clicked)

w.jslink((play, 'value'), (step_picker, 'value'))
w.jslink((slider_picker, 'value'), (step_picker, 'value'))
# w.jslink((play, 'interval'), (speed_picker, 'value'))
def change_speed(caller):
    play.interval = 600-(caller.new)*600+1
speed_picker.observe(change_speed, names='value')
display_box = w.VBox([w.HBox([meandemand, stddemand]), w.HBox([leadt, stdleadt]), w.HBox([reorderstock, reorderqty, review]),
                      runsimulation,
                      w.HBox([step_picker, slider_picker]),
                      w.HBox([play, speed_picker]),
                      out, m])


In [26]:
display_box